In [1]:
import numpy as np

In [2]:
# Define the nodal coordinates and element connectivity
coordinates = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
elements = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 3]])

In [3]:
# Define the material properties and cross-sectional area
E = 100e9
A = 0.01

In [4]:
def local_stiffness(A, E, x, y):
    L = np.sqrt((x[1] - x[0]) ** 2 + (y[1] - y[0]) ** 2)
    c = (x[1] - x[0]) / L
    s = (y[1] - y[0]) / L
    K_loc = (A * E / L) * np.array([[c**2,  c*s,   -c**2, -c*s], 
                                    [c*s,   s**2,  -c*s,  -s**2], 
                                    [-c**2, -c*s,  c**2,   c*s], 
                                    [-c*s,  -s**2, c*s,    s**2]])
    return K_loc

In [14]:
def stress(E, x0, y0, x, y):
    L = np.sqrt((x0[1] - x0[0]) ** 2 + (y0[1] - y0[0]) ** 2)
    c = (x0[1] - x0[0]) / L
    s = (y0[1] - y0[0]) / L
    sigma = E / L * (np.array([-c, -s, c, s]) @ np.array([x[0], y[0], x[1], y[1]]))
    return sigma

In [5]:
# Define the global stiffness matrix
n_nodes = len(coordinates)
n_elements = len(elements)
K_global = np.zeros((2 * n_nodes, 2 * n_nodes))
for i in range(n_elements):
    element = elements[i]
    x1, y1 = coordinates[element[0]]
    x2, y2 = coordinates[element[1]]
    K_e = local_stiffness(A, E, (x1, x2), (y1, y2))
    dof = np.array([2 * element[0], 2 * element[0] + 1, 2 * element[1], 2 * element[1] + 1])
    K_global[np.ix_(dof, dof)] += K_e

In [9]:
# Apply boundary conditions
fixed_dofs = np.array([0, 1, 4])     # dofs  1, 2 are fixed
free_dofs = np.setdiff1d(np.arange(2 * n_nodes), fixed_dofs)  # the rest of the dofs are free
K_ff = K_global[np.ix_(free_dofs, free_dofs)]
K_fe = K_global[np.ix_(free_dofs, fixed_dofs)]
K_ef = K_global[np.ix_(fixed_dofs, free_dofs)]
F = np.zeros(2 * n_nodes)
F[7] = -1000  # apply a downward force at dof 7
F_f = F[free_dofs]
F_e = F[fixed_dofs]

In [10]:
# Solve the resulting system of linear equations
U_f = np.linalg.solve(K_ff, F_f)
U = np.zeros(2 * n_nodes)
U[free_dofs] = U_f

In [11]:
# Calculate reactions
R_e = K_ef @ U_f
R_global = np.zeros(2 * n_nodes)
R_global[fixed_dofs] += R_e

In [26]:
# Calculate the element stresses and forces
sigma_e = np.zeros(n_elements)
N_e = np.zeros(n_elements)
for i in range(n_elements):
    element = elements[i]
    dof = np.array([2 * element[0], 2 * element[0] + 1, 2 * element[1], 2 * element[1] + 1])
    U_e = U[dof]    
    x01, y01 = coordinates[element[0]]
    x02, y02 = coordinates[element[1]]
    x1, y1 = U_e[:2]
    x2, y2 = U_e[2:]
    sigma_e[i] = stress(E, (x01, x02), (y01, y02), (x1, x2), (y1, y2))
    N_e[i] = A * sigma_e[i]

In [28]:
# Print the results
print("Displacements:")
print(U.reshape(-1, 2))
print("Reactions:")
print(R_e)
print("Element stress:")
print(sigma_e)
print("Element forces:")
print(N_e)

Displacements:
[[ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.82842712e-06]
 [ 0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-06 -3.82842712e-06]]
Reactions:
[ 1000.  1000. -1000.]
Element stress:
[      0.               0.         -141421.35623731       0.
  100000.        ]
Element forces:
[    0.             0.         -1414.21356237     0.
  1000.        ]
